In [1]:
pwd

'c:\\Users\\bhand\\OneDrive\\Desktop\\Deep Learning\\Nidarshan\\research'

In [2]:
cd ..

c:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan


In [3]:
pwd

'c:\\Users\\bhand\\OneDrive\\Desktop\\Deep Learning\\Nidarshan'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from Nidarshan.constants import *
from Nidarshan.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    
    def get_prepare_callbacks_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2023-12-10 22:12:27,037: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-12-10 22:12:27,037: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config
    
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-12-10 22:12:46,238: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan\config\config.yaml loaded successfully]
[2023-12-10 22:12:46,241: INFO: common: yaml file: C:\Users\bhand\OneDrive\Desktop\Deep Learning\Nidarshan\params.yaml loaded successfully]
[2023-12-10 22:12:46,241: INFO: common: Create directory at: artifacts]
[2023-12-10 22:12:46,241: INFO: common: Create directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-10 22:12:46,246: INFO: common: Create directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
